In [1]:
import socket
import time
import json
import threading
import pandas as pd
import re

#### This is a visualization of our DNS Zones

![zones_telematik.png](zones_telematik.png)

### This class is representing a DNS server and is handing all requests and responses to a DNS server

In [ ]:
def class DNS_server():
    
    socket = None
    address = None
    port = None
    is_authoharive = 0 #is this server an authorative server?
    is_root = 0        #is this server a root server?
    dns_zone = None    #own dns zone as json
    records_in_own_zone = None #all ns and a records in own zone. (this information is alreay 
                                    #conrained in the dns_zone variable but we are keeping this sepearate variable for now )
  
    def __init__(self, seconds=2000, , addr='127.0.0.9', port=53053): 
        self.address = addr
        self.port    = port
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        
        
    '''loads all entrys for own zone from our json file. 
    (e.g. zonename: telematik for zone (telematik, swith, router))'''
    def load_zone(zonename):
        ## load all entries for my zone
        with open('names.json') as dnsnamedata:
        data = json.load(dnsnamedata)
        self.dns_zone = data[0]
        
        telematik_dns_dict = {'ns': pd.DataFrame(self.dns_zone['ns'])['host'].tolist(),
                              'a':pd.DataFrame(self.dns_zone['a'])['value'].tolist()}
        self.records_in_own_zone = pd.DataFrame(telematik_dns_dict, columns = ['ns','a' ])
        
        print('loads the zones')
    
    '''checks if a requested entry is in own zone. 
    (e.g. this zonename is telematik and requested address is for dns name www.swith.telematik.
    Means that swith is in our zone and we can send the adress for switch.telematik)
    Returns None if not. '''
    def is_in_own_zone():
        names_i_know = pd.DataFrame(data[0]['ns'])['host'].tolist()
        
        print('check if x is in our zone')
        
        forward_to = None
    
        for name in names_i_know:
            #define pattern somestring.name
            pattern = re.compile(r'(.*?)(\.'+str(name)+')?$')

            #apply the addr we are looking for to the patter and return a list with results
            # e.g. ['www.switch', '.telematik'] for mattching or ['www.switch.telematik1', None] for not matching
            groups = pattern.match(addr).groups()

            if(groups[1] != None):
                forward_to = name

        return telematik_dns_df[telematik_dns_df['ns'] == forward_to]
    
    #check if this dns can send directly the authorative answer?
    #def have_authorative_answer():
        
    def response_to_client(payload, client_addr):
        
        try:
        payload['dns.flags.response'] = 'Changed!'
        payload_json = json.dumps(payload, indent = 4)
        time.sleep(0.5)
        server_socket.sendto(bytes(payload_json, 'utf-8'), client_addr)
    except:
        print('could not respond to client: ',client_addr)
        
    #def build_response():
    
    #this method is to start a thread to listen and handle requests and responses.
    def start():
        
        server_addr = (self.address, self.port)
        self.socket.bind(server_addr)
        
        
    
        